In [50]:
# Load tools needed for project 1

import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import panel as pn
import os
from pathlib import Path
from dotenv import load_dotenv
%matplotlib inline  

In [6]:
pn.extension('plotly')

In [7]:
import hvplot.pandas

In [35]:
load_dotenv()
mapbox = os.getenv("mapbox_api_key")
px.set_mapbox_access_token(mapbox)

In [36]:
# Set Variables

In [37]:
# Load Data


In [49]:
# Import Data

nfl_data = pd.DataFrame([])
years_start = 1996
years_end = 2022
for year in range(years_start,years_end):
        url = f'https://www.pro-football-reference.com/years/{year}/preseason_odds.htm'
        df = pd.read_html(url)[0]
        df['year'] = year
        nfl_data = nfl_data.append(df, ignore_index=True, sort=False)
        
nfl_data.columns = ['Team', 'Super Bowl Odds', 'W/l Over-Under', 'Record', 'Year']

nfl_data

# Strategy 1 - $100 on favorite
    
# Strategy 2 - $100 on Top 3
# Strategy 3 - $100 on Top 5

,Team,Super Bowl Odds,W/l Over-Under,Record,Year
0,Arizona Cardinals,12000,5.5,7-9 (over),1996
1,Atlanta Falcons,4000,8.0,3-13 (under),1996
2,Baltimore Ravens,3000,7.5,4-12 (under),1996
3,Buffalo Bills,1500,8.5,10-6 (over),1996
4,Carolina Panthers,8000,5.5,12-4 (over),1996
...,...,...,...,...,...
818,San Francisco 49ers,1400,10.5,10-7,2021
819,Seattle Seahawks,2500,9.5,7-10,2021
820,Tampa Bay Buccaneers,700,12.0,13-4,2021
821,Tennessee Titans,2000,9.5,12-5,2021


In [ ]:
# Charting - Individual Bar Graph Strat 1, 2, & 3

# Charting - Combined Bar Graph Total Wins Over Years

# Charting - Combine Bar Graph Total $ Won or Lost Over Years

In [ ]:
# Correlation Analysis

# 1. Correlation between end of season preseason leaders vs. Making SB
# Charting - Pie Chart? Scatter Plot?

In [ ]:
# 2. Correlation between end of season records of preseason favorites vs SB Teams
# Charting - Side by side - Bar Graph?

In [ ]:
# 3. Correlation between preseason wins vs regular season success  
# Charting - Side by side - Bar Graph?

In [ ]:
# Post Season Analysis
# Distance Travel by Teams wins and losses
# Charting - Heatmap?

# Post Season odds